## What the important is not the arrival point, it is the journey itself. Enjoy your journey :)

# Including 
* Single-Particle Theta and Real Space Hopping Phases
* Essential First Band Approximation Functions

In [1]:
using NBInclude; using LinearAlgebra
@nbinclude("Hofstadter Single Particle in Theta Space.ipynb")
@nbinclude(joinpath("..","First Band Approximation Functions.ipynb"));

# Initial Parameters

In [2]:
# Lattice Size
Nx=3; Ny=3; N=Nx*Ny

# Magnetic Flux Per Plaquet
p=1; q=Ny; alpha=p/q

# Total Particle Number
# NON-INTERACTING CASE:
PN = 1

# INTERACTING CASE
# PN = [0, 1, 2]

# Cut-Off for Sub-Space (Full Space for now...)
cut_off = Nx*Ny

# Interaction Potential
U = 2

# Theta Parameter Space
T_size = 5;

# Single Particle Operator

In [3]:
function Sp_Theta_Op(Nx, Ny, alpha, sp_basis, Tx, Ty)
        
    H_T = HSP_T(Nx, Ny, alpha, Tx, Ty, 0)
    
    H = SparseOperator(sp_basis)
    
    for m in 1:N
        for n in 1:N
            H = H + H_T[m,n] * transition(sp_basis, m, n)
        end
    end
    
    return H
end

Sp_Theta_Op (generic function with 1 method)

# Sub-Space Many-Body

In [4]:
@nbinclude("Hofstadter MB in Julia.ipynb"; regex=r"#.*executeme")
function get_total_H(Nx, Ny, alpha, Tx, Ty, cut_off, PN)
    
    sp_basis = NLevelBasis(N)

    # Single Particle Operator
    H1 = Sp_Theta_Op(Nx, Ny, alpha, sp_basis, Tx, Ty)
    
    # Sub-States Go on to Cut-Off
    sub_states = get_sub_states(H1, cut_off)

    # Sub-Space Basis and Projection Operator
    basis_sub, P, Pt = get_projector_op(sub_states, sp_basis)

    # Single-Particle Sub-Space Operator
    H1_sub = get_subspace_op(H1, P, Pt)

    # Many-Body Sub-Space Bosonic Fock States
    states_mb = bosonstates(basis_sub, PN)     

    # Many-Body Sub-Space Basis
    basis_mb = ManyBodyBasis(basis_sub, states_mb)

    basis_cut_mb, basis_cut_sp = get_Bosonic_MB_Basis(cut_off,PN);
    
    # Kinetic Term of Many-Body Sub-Space Operator
    H1_MB = get_mb_op(basis_mb, basis_sub, H1_sub)
    H1cut = SparseOperator(basis_cut_mb)
    H1cut.data = H1_MB.data

    # Interaction Term of Many-Body Sub-Space Operator
    H_Int = Hubbard_Interaction(basis_cut_sp, basis_cut_mb, P, Pt, cut_off)
    
    return H1cut + H_Int
end

get_total_H (generic function with 1 method)

In [6]:
Tx = Ty = 0
H = get_total_H(Nx, Ny, alpha, Tx, Ty, cut_off, PN)
EE, VV = eigen(dense((H+H')/2).data)

Eigen{ComplexF64, Float64, Matrix{ComplexF64}, Vector{Float64}}
values:
9-element Vector{Float64}:
 -2.7320508075688723
 -2.7320508075688688
 -2.7320508075688683
  0.7320508075688776
  0.7320508075688784
  0.7320508075688794
  1.9999999999999996
  2.0000000000000004
  2.0000000000000018
vectors:
9×9 Matrix{ComplexF64}:
   -0.0212317+0.105641im     …  -8.88178e-16-1.66533e-16im
    -0.535475+0.834129im         7.63278e-17-3.1225e-17im
   -0.0245826+0.0726677im                0.0+5.55112e-17im
  3.14419e-17-1.1601e-16im      -8.32667e-17+1.249e-16im
 -1.84043e-17-3.84079e-17im      3.26128e-16-1.73472e-16im
 -3.11058e-16+4.31512e-16im  …    3.1225e-17+1.73472e-16im
 -1.13258e-16+1.61275e-16im         -0.46967+0.294088im
  6.59195e-17-5.30175e-17im        -0.751429-0.203004im
  4.45222e-18-0.0im                -0.295071-0.0im

# Exact Diagonalization

In [26]:
# using DataFrames

# function get_filtered_energies(pn, basis, H_total)
    
#     # Operator Form of Eigenstates
#     E, V = eigenstates(dense(dense((H_total+H_total')/2)))

#     # Energies According to Each Particle Number
#     PN_Energies = Array{Float64}(undef, length(E), 2)
#     for i in 1:length(E)
#         # Particle Number in Each State
#         PN_Energies[i] = round(expect(number(basis), V[i]))
        
#         # Energies in Each State
#         PN_Energies[i,2] = E[i]
#     end
    
#     # Fiter Process
#     Filtered_Energies = DataFrame(PN_Energies, :auto)
#     Filtered_Energies = filter(row -> (row.x1 == pn),  Filtered_Energies)
    
#     return Filtered_Energies
# end

In [27]:
# # Energies of Filtered Particle Number 
# pn = 1
# df = get_filtered_energies(pn, basis_cut_mb, H_total)
# filtered_energies = Matrix(df)[:,2]

In [28]:
# using Plots

# # Plots Bands
# scatter(1:length(filtered_energies), filtered_energies, legend=false, axis=nothing)

In [29]:
# # Eigen-States of Filtered Particle Number 
# function Filtered_Hubbard_States(filtered_energies, states)
#     number_of_states = length(filtered_energies)
#     return states[:,1:number_of_states]
# end

In [30]:
# pn = 1

# function get_eigenstates(basis_cut_mb, H_total, pn)
    
#     E, V = eigen(dense((H_total+H_total')/2).data)

#     # Sorting Again...
#     #V = V[:, sortperm(E)]
    
#     df = get_filtered_energies(pn, basis_cut_mb, H_total)
    
#     filtered_energies = Matrix(df)[:,2]
    
#     #filtered_states = Filtered_Hubbard_States(filtered_energies, V)
#     #filtered_states = dropcol(V, 4)
    
#     return filtered_energies, V
# end

# # Orthogonality 
# # filtered_energies, filtered_states = get_eigenstates(basis_cut_mb, H_total, pn)
# # for i in 1:N
# #     j = 9
# #     a = filtered_states[:,j]
# #     b = filtered_states[:,i]
# #     println(dot(a,b))
# # end
# Tx = Ty = 0
# H_total = get_total_H(Nx, Ny, alpha, sp_basis, Tx, Ty, cut_off, PN, basis_cut_sp, basis_cut_mb)
# get_eigenstates(basis_cut_mb, H_total, pn)[1]

# CHERN CALCULATION SECTION

In [12]:
# function findcol(M, col)                
#     @inbounds @views for c in axes(M, 2)
#        M[:,c] == col && return c       
#     end                                 
#     return nothing                      
# end
# col = [
#     1.0 + 0.0im;
#  0.0 + 0.0im;
#  0.0 + 0.0im;
#  0.0 + 0.0im;
#  0.0 + 0.0im;
#  0.0 + 0.0im;
#  0.0 + 0.0im;
#  0.0 + 0.0im;
#  0.0 + 0.0im;
#  0.0 + 0.0im
# ]
# dropcol(M::AbstractMatrix, j) = M[:, deleteat!(collect(axes(M, 2)), j)];

In [31]:
# Mesh Grid for Theta Space Parameter
dx=2*pi/T_size; dy=dx;
Tx=range(start=0, stop=2*pi-dx, step=dx)
Ty=range(start=0, stop=2*pi-dy, step=dy);

In [66]:
function Chern_Nums(n1, n2)
    
    Sum=0
    
    for tx in Tx
        for ty in Ty
            
            H = get_total_H(Nx, Ny, alpha, tx, ty, cut_off, PN)
            
            w1, v1 = eigen(dense((H+H')/2).data)
            
            v1 = v1[:, sortperm(w1)]
            
            if isapprox(v1[1,1], 0+0im, atol=0.001) == false
                v1 = v1 ./ v1[1,1]
                v1 = v1/norm(v1)
            end

            # col_idx = findcol(v1, col)
            # v1 = dropcol(v1, col_idx)
            
            v1 = v1[:,n1:n2]
            
            #------------------------------------
            
            H = get_total_H(Nx, Ny, alpha, tx+dx, ty, cut_off, PN)
            
            w2, v2 = eigen(dense((H+H')/2).data)
            
            v2 = v2[:, sortperm(w2)]

            if isapprox(v2[1,1], 0+0im, atol=0.001) == false
                v2 = v2 ./ v2[1,1]
                v2 = v2 / norm(v2)
            end

            # col_idx = findcol(v2, col)
            # v2 = dropcol(v2, col_idx)
            
            v2 = v2[:,n1:n2]
            
            #------------------------------------
            
            H = get_total_H(Nx, Ny, alpha, tx, ty+dy, cut_off, PN)
            
            w3, v3 = eigen(dense((H+H')/2).data)
            
            v3 = v3[:, sortperm(w3)]

            if isapprox(v3[1,1], 0+0im, atol=0.001) == false
                v3 = v3 ./ v3[1,1]
                v3 = v3/norm(v3)
            end

            # col_idx = findcol(v3, col)
            # v3 = dropcol(v3, col_idx)
            
            v3 = v3[:,n1:n2]
            
            #------------------------------------
            
            H = get_total_H(Nx, Ny, alpha, tx+dx, ty+dy, cut_off, PN)
            
            w4, v4 = eigen(dense((H+H')/2).data)
            
            v4 = v4[:, sortperm(w4)]

            if isapprox(v4[1,1], 0+0im, atol=0.001) == false
                v4 = v4 ./ v4[1,1]
                v4 = v4/norm(v4)
            end

            # col_idx = findcol(v4, col)
            # v4 = dropcol(v4, col_idx)
            
            v4 = v4[:,n1:n2]
            
            #----------LINK VARIABLES------------
            
            U1=det(adjoint(v1)*v2)
            U1=U1/abs(U1)
            
            U2=det(adjoint(v2)*v4)
            U2=U2/abs(U2)
            
            U3=det(adjoint(v3)*v4)
            U3=U3/abs(U3)
            
            U4=det(adjoint(v1)*v3)
            U4=U4/abs(U4)
            
            #----------BERRY CURVATURE-----------
            
            F=log(U1*U2*1/U3*1/U4)
            Sum=Sum+F
            
        end
    end
    
    C = 1/(2*pi*1im)*Sum
    
    return C
end

Chern_Nums (generic function with 1 method)

In [67]:
Chern_Nums(1, 3)

-1.0792997342361128e-18 - 1.7669748230352825e-17im

# Check with "Hofstadter MB Chern" file

In [69]:
n1 = 1
n2 = 9
tx = ty = 0

0

In [71]:
H = get_total_H(Nx, Ny, alpha, tx, ty, cut_off, PN)

w1, v1 = eigen(dense((H+H')/2).data)

v1 = v1[:, sortperm(w1)]

if isapprox(v1[1,1], 0+0im, atol=0.001) == false
    v1 = v1 ./ v1[1,1]
    v1 = v1/norm(v1)
end

# col_idx = findcol(v1, col)
# v1 = dropcol(v1, col_idx)

v1 = v1[:,n1:n2]

#------------------------------------

H = get_total_H(Nx, Ny, alpha, tx+dx, ty, cut_off, PN)

w2, v2 = eigen(dense((H+H')/2).data)

v2 = v2[:, sortperm(w2)]

if isapprox(v2[1,1], 0+0im, atol=0.001) == false
    v2 = v2 ./ v2[1,1]
    v2 = v2 / norm(v2)
end

# col_idx = findcol(v2, col)
# v2 = dropcol(v2, col_idx)

v2 = v2[:,n1:n2]

#------------------------------------

H = get_total_H(Nx, Ny, alpha, tx, ty+dy, cut_off, PN)

w3, v3 = eigen(dense((H+H')/2).data)

v3 = v3[:, sortperm(w3)]

if isapprox(v3[1,1], 0+0im, atol=0.001) == false
    v3 = v3 ./ v3[1,1]
    v3 = v3/norm(v3)
end

# col_idx = findcol(v3, col)
# v3 = dropcol(v3, col_idx)

v3 = v3[:,n1:n2]

#------------------------------------

H = get_total_H(Nx, Ny, alpha, tx+dx, ty+dy, cut_off, PN)

w4, v4 = eigen(dense((H+H')/2).data)

v4 = v4[:, sortperm(w4)]

if isapprox(v4[1,1], 0+0im, atol=0.001) == false
    v4 = v4 ./ v4[1,1]
    v4 = v4/norm(v4)
end

# col_idx = findcol(v4, col)
# v4 = dropcol(v4, col_idx)

v4 = v4[:,n1:n2]

#----------LINK VARIABLES------------

U1=det(adjoint(v1)*v2)
U1=U1/abs(U1)

U2=det(adjoint(v2)*v4)
U2=U2/abs(U2)

U3=det(adjoint(v3)*v4)
U3=U3/abs(U3)

U4=det(adjoint(v1)*v3)
U4=U4/abs(U4)

#----------BERRY CURVATURE-----------

F=log(U1*U2*1/U3*1/U4)

8.52741319615994e-32 - 4.129748950277714e-16im

In [73]:
eigen(v1)

Eigen{ComplexF64, ComplexF64, Matrix{ComplexF64}, Vector{ComplexF64}}
values:
9-element Vector{ComplexF64}:
 -0.33171144944039976 + 0.0328424332420734im
 -0.32944251696367644 - 0.050781287180897416im
 -0.31160750339306503 + 0.11837176580693569im
  -0.2689834502969823 + 0.19687309256839067im
 -0.16947438557723316 - 0.2870357882639909im
  0.09547931048700889 + 0.31936626681607505im
   0.2237484943774718 - 0.24707837294050172im
   0.3326247298839208 + 0.02172326358446125im
  0.33331581880936106 - 0.003417022469445724im
vectors:
9×9 Matrix{ComplexF64}:
    0.0773191+0.511891im     …   3.08526e-16+1.62928e-19im
     0.129845-0.56231im          1.64921e-16-1.29095e-16im
     0.631615+0.0im             -1.48585e-16-1.38088e-16im
  -2.3032e-16-4.75835e-16im         0.751813+0.0im
 -2.09388e-17-1.91545e-16im        -0.104663-0.021228im
 -1.77055e-16+3.33217e-16im  …      0.243715+0.603304im
 -4.12767e-16+3.11066e-16im       3.0247e-15-3.14179e-15im
 -2.24324e-16-1.04232e-16im      4.97207e-15-2

In [63]:
PN = 1
tx = 1
ty = 1
#---
H = get_total_H(Nx, Ny, alpha, Tx[tx], Ty[ty], cut_off, PN)     
w1, v1 = eigen(dense((H+H')/2).data)
v1 = v1[:, sortperm(w1)]
# drop_idx = findcol(v1, col)
# v1 = dropcol(v1, drop_idx)
if isapprox(v1[1,1], 0+0im, atol=0.001) == false
    v1 = v1 ./ v1[1,1]
    v1 = v1/norm(v1)
end
v1 = v1[:,n1:n2]
#---
H = get_total_H(Nx, Ny, alpha, Tx[tx]+dx, Ty[ty], cut_off, PN)     
w2, v2 = eigen(dense((H+H')/2).data)
v2 = v2[:, sortperm(w2)]
# drop_idx = findcol(v2, col)
# v2 = dropcol(v2, drop_idx)
v2 = v2[:,n1:n2]
#---
H = get_total_H(Nx, Ny, alpha, Tx[tx], Ty[ty]+dy, cut_off, PN)     
w3, v3 = eigen(dense((H+H')/2).data)
v3 = v3[:, sortperm(w3)]
# drop_idx = findcol(v3, col)
# v3 = dropcol(v3, drop_idx)
v3 = v3[:,n1:n2]
#---
H = get_total_H(Nx, Ny, alpha, Tx[tx]+dx, Ty[ty]+dy, cut_off, PN)     
w4, v4 = eigen(dense((H+H')/2).data)
v4 = v4[:, sortperm(w4)]
# drop_idx = findcol(v4, col)
# v4 = dropcol(v4, drop_idx)
v4 = v4[:,n1:n2]
#---
U1=det(adjoint(v1)*v2)
U1=U1/abs(U1)
U2=det(adjoint(v2)*v4)
U2=U2/abs(U2)
U3=det(adjoint(v3)*v4)
U3=U3/abs(U3)
U4=det(adjoint(v1)*v3)
U4=U4/abs(U4)

0.09998746114914639 - 0.9949886972287413im

In [64]:
w4

9-element Vector{Float64}:
 -2.546639394327162
 -2.5466393943271606
 -2.546639394327156
  0.20750036611082465
  0.20750036611082553
  0.20750036611082567
  2.339139028216335
  2.339139028216336
  2.3391390282163362

In [65]:
v1[:,2:2]

9×1 Matrix{ComplexF64}:
    0.22768685118156337 - 0.09547546849024292im
   -0.09486724412515558 - 0.029938605636929395im
     0.9486625031502366 + 0.17084049202965787im
 1.3877787807814457e-17 + 9.71445146547012e-17im
 -9.020562075079397e-17 + 1.5612511283791264e-17im
   2.42861286636753e-17 - 6.938893903907228e-18im
 -1.734723475976807e-16 + 1.734723475976807e-16im
                    0.0 + 9.367506770274758e-17im
  3.354703270462921e-16 - 0.0im

In [53]:
A = [0.04206576838675055 - 0.4844396754976562im
  -0.11575260100874928 - 0.5048774472308879im
 -0.019490509012952906 - 0.41988073170016393im
   0.08258610549150286 - 0.15124676786746452im
  -0.13712915132022854 - 0.206737076416468im
   0.02043777173323122 - 0.1578183693955im
    0.1675828210222271 - 0.2475088598632606im
  -0.20168809511772048 - 0.2682933538161717im
                   0.0 + 0.0im]

9-element Vector{ComplexF64}:
   0.04206576838675055 - 0.4844396754976562im
  -0.11575260100874928 - 0.5048774472308879im
 -0.019490509012952906 - 0.41988073170016393im
   0.08258610549150286 - 0.15124676786746452im
  -0.13712915132022854 - 0.206737076416468im
   0.02043777173323122 - 0.1578183693955im
    0.1675828210222271 - 0.2475088598632606im
  -0.20168809511772048 - 0.2682933538161717im
                   0.0 + 0.0im

In [58]:
v1[:,4:4] ./ A

9×1 Matrix{ComplexF64}:
 -1.7980148799796905e-16 + 2.6448446692249435e-16im
  3.1046229679841827e-16 + 8.476994294114351e-16im
 -2.5323280801083318e-17 + 1.227685310490113e-16im
      -1.090534299951408 - 0.4449238617579872im
     0.36100073604058036 + 0.09170231699197172im
     -2.7721760625514626 - 5.462498847891495im
   8.007269689386028e-17 + 1.7066629704076275e-16im
   8.099994278116515e-17 - 7.990563131415732e-16im
                     NaN + NaN*im

In [49]:
eigen(v1)

Eigen{ComplexF64, ComplexF64, Matrix{ComplexF64}, Vector{ComplexF64}}
values:
9-element Vector{ComplexF64}:
  -0.9957553612966031 + 0.09203945050397394im
 -0.42003967852213075 - 0.9075057401840629im
  -0.2605125273634965 + 0.9654704672265649im
 -0.18697876421055862 + 0.9823639558403457im
 -0.16395678044357687 - 0.9864675231078723im
  0.09948476278099364 - 0.9950390856516179im
  0.34409654116713523 - 0.9389342737150541im
   0.5944419686309548 + 0.8041385116571351im
   0.9444051792358245 - 0.32878390689410203im
vectors:
9×9 Matrix{ComplexF64}:
    -0.503198-0.224812im     …   1.60054e-16-8.15074e-17im
    -0.494572+0.291905im         9.05782e-17+7.01495e-17im
     0.605344+0.0im             -1.78743e-16-8.81097e-18im
  4.41479e-16+3.9254e-17im      -2.13933e-16+1.97689e-16im
 -1.69026e-17+7.64161e-17im      2.57639e-16-5.272e-16im
  8.06173e-17+2.73893e-17im  …   1.03897e-16+2.52044e-16im
  1.17364e-16+1.10632e-16im        -0.596552+0.306689im
  3.10486e-16+2.09361e-16im         0.716812